# Concurrent Orchestration Pattern with Semantic Kernel Agents

This notebook demonstrates the concurrent orchestration pattern using Semantic Kernel agents. Concurrent orchestration is a design approach where multiple agents execute tasks simultaneously, enabling efficient handling of complex workflows. By leveraging Azure OpenAI services, these agents can provide intelligent and context-aware responses.

### Key Features of Concurrent Orchestration:
- **Parallel Execution**: Agents work independently on their assigned tasks, reducing overall execution time.
- **Task Specialization**: Each agent is configured with specific expertise, ensuring high-quality outputs.
- **Error Handling**: Robust mechanisms are implemented to manage initialization and runtime errors.
- **Concise Responses**: Agents are instructed to provide brief, two-sentence answers for clarity and focus.

### Examples in This Notebook:
1. **Scientific Question**: "What is temperature?" - Agents collaborate to provide concise answers from different scientific perspectives.
2. **Practical Task**: "How to chop vegetables efficiently and safely?" - Agents offer practical advice on knife skills and safety measures.

The notebook utilizes `ConcurrentOrchestration`, `ChatCompletionAgent`, and `InProcessRuntime` to manage and execute tasks concurrently, showcasing the power and flexibility of Semantic Kernel.

We will:
1. Define agents for cooking tasks.
2. Set up concurrent orchestration.
3. Invoke the orchestration and collect results.

In [1]:
# Import Required Libraries
from semantic_kernel.agents import ChatCompletionAgent, ConcurrentOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

**Expected Output**: Package installation messages followed by successful import confirmation. No output if imports are successful.

In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Define Agents for Chopping Vegetables
azure_service = AzureChatCompletion(
    api_key="YOUR_AZURE_OPENAI_API_KEY",
    endpoint="YOUR_AZURE_OPENAI_ENDPOINT",
    deployment_name="YOUR_DEPLOYMENT_NAME"
)

knife_skills_agent = ChatCompletionAgent(
    name="KnifeSkillsExpert",
    instructions="Provide a concise explanation of the best techniques for chopping vegetables in 2 sentences.",
    service=azure_service
)

safety_agent = ChatCompletionAgent(
    name="SafetyExpert",
    instructions="Provide a concise explanation of safety tips for chopping vegetables in 2 sentences.",
    service=azure_service
)

# Set Up Concurrent Orchestration
agents = [knife_skills_agent, safety_agent]
concurrent_orchestration = ConcurrentOrchestration(members=agents)

# Start the Runtime
runtime = InProcessRuntime()
runtime.start()

# Invoke the Orchestration and Collect Results
try:
    orchestration_result = await concurrent_orchestration.invoke(
        task="How to chop vegetables efficiently and safely?",
        runtime=runtime
    )

    # Collect results
    value = await orchestration_result.get(timeout=30)
    for item in value:
        print(f"# {item.name}: {item.content}")  # Access attributes directly
except Exception as e:
    print(f"Error during orchestration: {e}")
finally:
    # Stop the runtime
    await runtime.stop_when_idle()

# SafetyExpert: To chop vegetables efficiently and safely, always use a sharp knife and a stable cutting board to prevent slipping. Keep your fingertips tucked under (using a "claw grip") to protect them from the blade and work at a steady, controlled pace.
# KnifeSkillsExpert: To chop vegetables efficiently and safely, use a sharp knife and employ the "claw grip," tucking your fingertips under while holding the vegetable to protect them. Keep the knife tip on the cutting surface while using smooth rocking motions, and ensure a stable cutting board to prevent slipping.


**Expected Output**: Messages showing successful creation of cooking expert agents, orchestration setup, and runtime initialization. You'll see confirmation that the concurrent system is ready.

In [ ]:
# Import Required Libraries
from semantic_kernel.agents import ChatCompletionAgent, ConcurrentOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
import time

def get_agents():
    try:
        # Azure OpenAI service configuration
        azure_service = AzureChatCompletion(
            api_key="YOUR_AZURE_OPENAI_API_KEY",
            endpoint="YOUR_AZURE_OPENAI_ENDPOINT",
            deployment_name="YOUR_DEPLOYMENT_NAME"
        )

        physics_agent = ChatCompletionAgent(
            name="PhysicsExpert",
            instructions="Provide a concise explanation of temperature from a physics perspective in 2 sentences.",
            service=azure_service
        )

        chemistry_agent = ChatCompletionAgent(
            name="ChemistryExpert",
            instructions="Provide a concise explanation of temperature from a chemistry perspective in 2 sentences.",
            service=azure_service
        )

        return [physics_agent, chemistry_agent]

    except Exception as e:
        print(f"Error initializing agents: {e}")
        print("Retrying in 5 seconds...")
        time.sleep(5)
        return get_agents()

agents = get_agents()  
concurrent_orchestration = ConcurrentOrchestration(members=agents)

# Start the runtime
runtime = InProcessRuntime()
runtime.start()

# Invoke the Orchestration and Collect Results
try:
    orchestration_result = await concurrent_orchestration.invoke(
        task="What is temperature?",
        runtime=runtime
    )

    # Collect results
    value = await orchestration_result.get(timeout=30)
    for item in value:
        print(f"# {item.name}: {item.content}")  # Access attributes directly
except Exception as e:
    print(f"Error during orchestration: {e}")
finally:
    # Stop the runtime
    await runtime.stop_when_idle()

# ChemistryExpert: From a chemistry perspective, temperature is a measure of the average kinetic energy of the particles in a substance. It reflects how fast the particles (atoms or molecules) are moving or vibrating, influencing reaction rates and phase changes.
# PhysicsExpert: In physics, temperature is a measure of the average kinetic energy of the particles in a substance, representing how fast the particles are moving or vibrating. It determines the direction of heat transfer, flowing from higher to lower temperature regions.


**Expected Output**: Results from both agents working simultaneously on the same task. You'll see responses from multiple expert agents (like PhysicsExpert and ChemistryExpert) providing their perspectives concurrently, demonstrating parallel processing.